In [1]:
from gernerate_data import load_clas_seg_data
import tensorflow as tf
from sklearn.metrics import classification_report, auc, roc_curve
from keras.utils.np_utils import *
from keras.callbacks import LearningRateScheduler
from models.shareMoreLayers_sobel import create_pair_model
from sklearn.preprocessing import LabelBinarizer, label_binarize
from utils.losses import dice_coef_loss, dice_coef, dice_p_bce, dice_p_focal, tversky_loss, focal_loss, focal_tversky, \
    p_r_f1_iou, generalized_dice_coeff, generalized_dice_loss
from keras.losses import categorical_crossentropy, mean_squared_error, binary_crossentropy
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, CSVLogger
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import keras.backend as K
import utils_paths
import numpy as np
import pickle
import os

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [2]:
INIT_LR = 1e-3
EPOCHS =300
batch_size = 8
depth = 3
img_size = 224
Name = "shareLayer"
GPU = True
target = (img_size, img_size)

if GPU:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "2"


    
print("------------------------------------------------ Preparing model ------------------------------------------------")
model = create_pair_model(img_size, img_size, depth, nClasses=2)
model.summary()

------------------------------------------------ Preparing model ------------------------------------------------
Instructions for updating:
Colocations handled automatically by placer.


2022-03-21 01:12:21.669169: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-03-21 01:12:21.705301: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200095000 Hz
2022-03-21 01:12:21.709144: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x4666460 executing computations on platform Host. Devices:
2022-03-21 01:12:21.709235: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2022-03-21 01:12:22.346187: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x46e2d20 executing computations on platform CUDA. Devices:
2022-03-21 01:12:22.346278: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla V100-PCIE-32GB, Compute Capability 7.0
2022-03-21 01:12:22.347185: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found device 0 with properties: 
name: Tes

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cls_input (InputLayer)          (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
seg_input (InputLayer)          (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 224, 224, 3)  0           cls_input[0][0]                  
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 224, 224, 3)  0           seg_input[0][0]             

In [3]:
# trainX_dir = 'dataset/LE/test/images/'
trainX_dir = 'dataset/Dataset_BUSI_AN/train/images/'
valX_dir = trainX_dir.replace('train', 'val')
testX_dir = trainX_dir.replace('train', 'test')


train_x,  train_c_y, train_s_y = load_clas_seg_data(trainX_dir, target)
val_x, val_c_y, val_s_y = load_clas_seg_data(valX_dir, target)
test_x, test_c_y, test_s_y = load_clas_seg_data(testX_dir, target)


lb = LabelBinarizer()
train_c_y = lb.fit_transform(train_c_y)
val_c_y = lb.fit_transform(val_c_y)
test_c_y = lb.fit_transform(test_c_y)
train_c_y = to_categorical(train_c_y, 2)
val_c_y = to_categorical(val_c_y, 2)
test_c_y = to_categorical(test_c_y, 2)

dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (164).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (164).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (387)_6f63a5f8-c4f5-4ee9-a5d1-339421652c50.png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (387)_6f63a5f8-c4f5-4ee9-a5d1-339421652c50.png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (376)_6e0b291d-45e3-4ff8-aa59-65d83b825bdc.png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (376)_6e0b291d-45e3-4ff8-aa59-65d83b825bdc.png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (137).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (137).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (240).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (240).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (428).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (428).png
dataset/Dataset_BUSI_AN/train/images/bengin_imag

dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (79)_517e6812-ff60-48df-8410-56a55f2b8b4d.png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (79)_517e6812-ff60-48df-8410-56a55f2b8b4d.png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (52)_f35bb3b1-4c49-489e-908d-cbcd5c78addc.png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (52)_f35bb3b1-4c49-489e-908d-cbcd5c78addc.png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (325).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (325).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (27)_52c79025-6a96-4b9e-a4af-e88911075f09.png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (27)_52c79025-6a96-4b9e-a4af-e88911075f09.png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (386).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (386).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (269)_fdce7b8d-fbfe-4fad-9f50-d447e4f3ca16.png
dataset/D

dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (255).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (255).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (406).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (406).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (134).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (134).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (139).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (139).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (368).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (368).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (293)_936fc5f4-c397-4b3a-93a9-b16e445e817c.png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (293)_936fc5f4-c397-4b3a-93a9-b16e445e817c.png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (165).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benig

dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (156).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (156).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (5).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (5).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (383).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (383).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (6).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (6).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (354).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (354).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (433).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (433).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (132).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (132).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (2).png
data

dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (175).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (175).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (175)_1b69ec69-1f58-46ed-88be-76582d7393b0.png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (175)_1b69ec69-1f58-46ed-88be-76582d7393b0.png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (277).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (277).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (162).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (162).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (182).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (182).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (46).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (46).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (54)_cda37892-1794-4e34-9f21-dfda13a340aa.png
dataset/Dataset_BUSI

...

dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (43)_e23f9e9b-7741-444e-9c7d-6ef56685c7a0.png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (43)_e23f9e9b-7741-444e-9c7d-6ef56685c7a0.png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (151)_14dfad26-99f8-4390-9a5f-1200f796b0de.png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (151)_14dfad26-99f8-4390-9a5f-1200f796b0de.png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (32).png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (32).png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (78).png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (78).png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (115).png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (115).png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (116)_693b0b79-8890-4702-92f8-1188b572a17c.png
dataset/Dataset_B

dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (31).png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (31).png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (128)_c7218a00-dd38-4ee4-a011-ea9d80f4bddb.png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (128)_c7218a00-dd38-4ee4-a011-ea9d80f4bddb.png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (51)_1452949f-f862-43c3-95e5-fc802578fbd9.png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (51)_1452949f-f862-43c3-95e5-fc802578fbd9.png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (67)_080e2107-b37f-4e23-bdd0-bbecdcdb1e76.png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (67)_080e2107-b37f-4e23-bdd0-bbecdcdb1e76.png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (87)_603a10bb-e3c7-42f8-bc31-2ba92f638c32.png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (87)_603a10bb-e3c7-42f8-bc31

dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (130)_fc09bae4-341a-44c3-84f2-eb3d1dd59921.png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (130)_fc09bae4-341a-44c3-84f2-eb3d1dd59921.png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (62)_3b4406bb-d01a-4565-b536-a515a52f95da.png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (62)_3b4406bb-d01a-4565-b536-a515a52f95da.png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (111)_80bdb5f2-021d-4130-a452-cb1508b7501c.png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (111)_80bdb5f2-021d-4130-a452-cb1508b7501c.png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (151)_1440aec6-9669-4d5f-ade0-5b45ddb93272.png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (151)_1440aec6-9669-4d5f-ade0-5b45ddb93272.png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (63).png
dataset/Dataset_BUSI_AN/train/masks/malignant_mas

dataset/Dataset_BUSI_AN/val/images/bengin_images/benign (138).png
dataset/Dataset_BUSI_AN/val/masks/bengin_masks/benign (138).png
dataset/Dataset_BUSI_AN/val/images/bengin_images/benign (334).png
dataset/Dataset_BUSI_AN/val/masks/bengin_masks/benign (334).png
dataset/Dataset_BUSI_AN/val/images/bengin_images/benign (284).png
dataset/Dataset_BUSI_AN/val/masks/bengin_masks/benign (284).png
dataset/Dataset_BUSI_AN/val/images/bengin_images/benign (311).png
dataset/Dataset_BUSI_AN/val/masks/bengin_masks/benign (311).png
dataset/Dataset_BUSI_AN/val/images/bengin_images/benign (98).png
dataset/Dataset_BUSI_AN/val/masks/bengin_masks/benign (98).png
dataset/Dataset_BUSI_AN/val/images/bengin_images/benign (242).png
dataset/Dataset_BUSI_AN/val/masks/bengin_masks/benign (242).png
dataset/Dataset_BUSI_AN/val/images/bengin_images/benign (78).png
dataset/Dataset_BUSI_AN/val/masks/bengin_masks/benign (78).png
dataset/Dataset_BUSI_AN/val/images/bengin_images/benign (304).png
dataset/Dataset_BUSI_AN/val/

dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (57).png
dataset/Dataset_BUSI_AN/test/masks/bengin_masks/benign (57).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (220).png
dataset/Dataset_BUSI_AN/test/masks/bengin_masks/benign (220).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (103).png
dataset/Dataset_BUSI_AN/test/masks/bengin_masks/benign (103).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (150).png
dataset/Dataset_BUSI_AN/test/masks/bengin_masks/benign (150).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (84).png
dataset/Dataset_BUSI_AN/test/masks/bengin_masks/benign (84).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (88).png
dataset/Dataset_BUSI_AN/test/masks/bengin_masks/benign (88).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (391).png
dataset/Dataset_BUSI_AN/test/masks/bengin_masks/benign (391).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (408).png
dataset/Dataset

In [4]:
# define callbacks
csv_logger = CSVLogger(Name+'.log')

reduce_lr = ReduceLROnPlateau(monitor='classification_output_loss', factor=0.3, patience=20, min_lr=1e-8, mode='auto', verbose=1)

# checkpoint_period1 = ModelCheckpoint(Name + '-{epoch:03d}-{classification_output_acc:.4f}.h5',
#                                      monitor='classification_output_acc', mode='auto', save_best_only='True')



# define loss and compile the model

opt = Adam(lr=INIT_LR, beta_1=0.9, beta_2=0.99, epsilon=1e-08, decay=0.01)

def binary_crossentropy(y_true, y_pred):
    e=1.0
    return K.mean(-(y_true*K.log(y_pred+K.epsilon())+
                    e*(1-y_true)*K.log(1-y_pred+K.epsilon())),
                  axis=-1)

def tversky(y_true, y_pred, smooth=1):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1 - y_pred_pos))
    false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
    alpha = 0.5
    return (true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)

def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true, y_pred)


from keras.losses import categorical_crossentropy

model.compile(loss={'segmentation_output': 'categorical_crossentropy', "classification_output": binary_crossentropy},
              loss_weights={'segmentation_output': 0.5, "classification_output": 0.5},
              optimizer=opt, 
              metrics={'segmentation_output':['accuracy', dice_coef], 'classification_output':'accuracy'})


In [5]:

print("================================================ begin training ================================================")

hist = model.fit([train_x,train_x], 
                 [train_c_y, train_s_y],
                 batch_size=batch_size,
                 epochs=200,
                 validation_data=([val_x,val_x], [val_c_y, val_s_y]),
                 verbose=1,
                 callbacks=[reduce_lr, csv_logger],
                 shuffle=True)

print("================================================ Saving model ================================================")
model_filename = "shareLayer_Sobel.h5"
model.save_weights(model_filename)
print('model saved to:', model_filename)

================================================ begin training ================================================
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 1000 samples, validate on 64 samples
Epoch 1/200


2022-03-21 01:16:29.440701: I tensorflow/stream_executor/dso_loader.cc:152] successfully opened CUDA library libcublas.so.10.0 locally


1000/1000 [==============================] - 48s 48ms/step - loss: 0.6351 - classification_output_loss: 0.9326 - segmentation_output_loss: 0.3377 - classification_output_acc: 0.5820 - segmentation_output_acc: 0.8762 - segmentation_output_dice_coef: 0.7966 - val_loss: 0.4309 - val_classification_output_loss: 0.6168 - val_segmentation_output_loss: 0.2451 - val_classification_output_acc: 0.7031 - val_segmentation_output_acc: 0.9149 - val_segmentation_output_dice_coef: 0.8340
Epoch 2/200
1000/1000 [==============================] - 29s 29ms/step - loss: 0.4770 - classification_output_loss: 0.6680 - segmentation_output_loss: 0.2860 - classification_output_acc: 0.5710 - segmentation_output_acc: 0.8863 - segmentation_output_dice_coef: 0.8247 - val_loss: 0.4397 - val_classification_output_loss: 0.6675 - val_segmentation_output_loss: 0.2118 - val_classification_output_acc: 0.5156 - val_segmentation_output_acc: 0.9178 - val_segmentation_output_dice_coef: 0.8706
Epoch 3/200
1000/1000 [===========

Epoch 18/200
1000/1000 [==============================] - 52s 52ms/step - loss: 0.1918 - classification_output_loss: 0.2773 - segmentation_output_loss: 0.1064 - classification_output_acc: 0.9470 - segmentation_output_acc: 0.9593 - segmentation_output_dice_coef: 0.9360 - val_loss: 0.2317 - val_classification_output_loss: 0.2772 - val_segmentation_output_loss: 0.1862 - val_classification_output_acc: 0.9375 - val_segmentation_output_acc: 0.9462 - val_segmentation_output_dice_coef: 0.9296
Epoch 19/200
1000/1000 [==============================] - 53s 53ms/step - loss: 0.1884 - classification_output_loss: 0.2770 - segmentation_output_loss: 0.0999 - classification_output_acc: 0.9430 - segmentation_output_acc: 0.9615 - segmentation_output_dice_coef: 0.9399 - val_loss: 0.2249 - val_classification_output_loss: 0.2523 - val_segmentation_output_loss: 0.1975 - val_classification_output_acc: 0.9531 - val_segmentation_output_acc: 0.9430 - val_segmentation_output_dice_coef: 0.9342
Epoch 20/200
1000/10

Epoch 35/200
1000/1000 [==============================] - 54s 54ms/step - loss: 0.1255 - classification_output_loss: 0.2109 - segmentation_output_loss: 0.0402 - classification_output_acc: 0.9620 - segmentation_output_acc: 0.9834 - segmentation_output_dice_coef: 0.9744 - val_loss: 0.2329 - val_classification_output_loss: 0.2443 - val_segmentation_output_loss: 0.2215 - val_classification_output_acc: 0.9375 - val_segmentation_output_acc: 0.9467 - val_segmentation_output_dice_coef: 0.9396
Epoch 36/200
1000/1000 [==============================] - 53s 53ms/step - loss: 0.1183 - classification_output_loss: 0.1984 - segmentation_output_loss: 0.0383 - classification_output_acc: 0.9730 - segmentation_output_acc: 0.9840 - segmentation_output_dice_coef: 0.9756 - val_loss: 0.2531 - val_classification_output_loss: 0.2732 - val_segmentation_output_loss: 0.2330 - val_classification_output_acc: 0.9219 - val_segmentation_output_acc: 0.9414 - val_segmentation_output_dice_coef: 0.9319
Epoch 37/200
1000/10

...

Epoch 86/200
1000/1000 [==============================] - 48s 48ms/step - loss: 0.0996 - classification_output_loss: 0.1865 - segmentation_output_loss: 0.0126 - classification_output_acc: 0.9730 - segmentation_output_acc: 0.9935 - segmentation_output_dice_coef: 0.9907 - val_loss: 0.2643 - val_classification_output_loss: 0.2520 - val_segmentation_output_loss: 0.2765 - val_classification_output_acc: 0.9531 - val_segmentation_output_acc: 0.9498 - val_segmentation_output_dice_coef: 0.9464
Epoch 87/200
1000/1000 [==============================] - 47s 47ms/step - loss: 0.1086 - classification_output_loss: 0.2045 - segmentation_output_loss: 0.0127 - classification_output_acc: 0.9730 - segmentation_output_acc: 0.9935 - segmentation_output_dice_coef: 0.9907 - val_loss: 0.2619 - val_classification_output_loss: 0.2465 - val_segmentation_output_loss: 0.2774 - val_classification_output_acc: 0.9375 - val_segmentation_output_acc: 0.9517 - val_segmentation_output_dice_coef: 0.9485
Epoch 88/200
1000/10

Epoch 103/200
1000/1000 [==============================] - 49s 49ms/step - loss: 0.0985 - classification_output_loss: 0.1863 - segmentation_output_loss: 0.0107 - classification_output_acc: 0.9680 - segmentation_output_acc: 0.9943 - segmentation_output_dice_coef: 0.9919 - val_loss: 0.3020 - val_classification_output_loss: 0.3208 - val_segmentation_output_loss: 0.2833 - val_classification_output_acc: 0.9531 - val_segmentation_output_acc: 0.9516 - val_segmentation_output_dice_coef: 0.9488
Epoch 104/200
1000/1000 [==============================] - 48s 48ms/step - loss: 0.1016 - classification_output_loss: 0.1932 - segmentation_output_loss: 0.0099 - classification_output_acc: 0.9690 - segmentation_output_acc: 0.9946 - segmentation_output_dice_coef: 0.9922 - val_loss: 0.3042 - val_classification_output_loss: 0.3202 - val_segmentation_output_loss: 0.2882 - val_classification_output_acc: 0.9531 - val_segmentation_output_acc: 0.9516 - val_segmentation_output_dice_coef: 0.9489
Epoch 105/200
1000

Epoch 136/200
1000/1000 [==============================] - 48s 48ms/step - loss: 0.1002 - classification_output_loss: 0.1915 - segmentation_output_loss: 0.0089 - classification_output_acc: 0.9590 - segmentation_output_acc: 0.9950 - segmentation_output_dice_coef: 0.9928 - val_loss: 0.2943 - val_classification_output_loss: 0.2970 - val_segmentation_output_loss: 0.2916 - val_classification_output_acc: 0.9531 - val_segmentation_output_acc: 0.9515 - val_segmentation_output_dice_coef: 0.9487
Epoch 137/200
1000/1000 [==============================] - 48s 48ms/step - loss: 0.1021 - classification_output_loss: 0.1950 - segmentation_output_loss: 0.0092 - classification_output_acc: 0.9660 - segmentation_output_acc: 0.9949 - segmentation_output_dice_coef: 0.9926 - val_loss: 0.2974 - val_classification_output_loss: 0.3027 - val_segmentation_output_loss: 0.2921 - val_classification_output_acc: 0.9531 - val_segmentation_output_acc: 0.9516 - val_segmentation_output_dice_coef: 0.9489
Epoch 138/200
1000

Epoch 153/200
1000/1000 [==============================] - 48s 48ms/step - loss: 0.0951 - classification_output_loss: 0.1808 - segmentation_output_loss: 0.0095 - classification_output_acc: 0.9790 - segmentation_output_acc: 0.9948 - segmentation_output_dice_coef: 0.9926 - val_loss: 0.3019 - val_classification_output_loss: 0.3071 - val_segmentation_output_loss: 0.2966 - val_classification_output_acc: 0.9531 - val_segmentation_output_acc: 0.9515 - val_segmentation_output_dice_coef: 0.9489
Epoch 154/200
1000/1000 [==============================] - 48s 48ms/step - loss: 0.0948 - classification_output_loss: 0.1806 - segmentation_output_loss: 0.0091 - classification_output_acc: 0.9750 - segmentation_output_acc: 0.9950 - segmentation_output_dice_coef: 0.9927 - val_loss: 0.2970 - val_classification_output_loss: 0.2990 - val_segmentation_output_loss: 0.2950 - val_classification_output_acc: 0.9531 - val_segmentation_output_acc: 0.9516 - val_segmentation_output_dice_coef: 0.9490
Epoch 155/200
1000


Epoch 00169: ReduceLROnPlateau reducing learning rate to 8.100000013655517e-06.
Epoch 170/200
1000/1000 [==============================] - 48s 48ms/step - loss: 0.0937 - classification_output_loss: 0.1785 - segmentation_output_loss: 0.0089 - classification_output_acc: 0.9650 - segmentation_output_acc: 0.9950 - segmentation_output_dice_coef: 0.9928 - val_loss: 0.3051 - val_classification_output_loss: 0.3144 - val_segmentation_output_loss: 0.2958 - val_classification_output_acc: 0.9531 - val_segmentation_output_acc: 0.9518 - val_segmentation_output_dice_coef: 0.9491
Epoch 171/200
1000/1000 [==============================] - 48s 48ms/step - loss: 0.0969 - classification_output_loss: 0.1850 - segmentation_output_loss: 0.0088 - classification_output_acc: 0.9730 - segmentation_output_acc: 0.9951 - segmentation_output_dice_coef: 0.9929 - val_loss: 0.3047 - val_classification_output_loss: 0.3131 - val_segmentation_output_loss: 0.2963 - val_classification_output_acc: 0.9531 - val_segmentation_

In [1]:
N = np.arange(0, 200)
plt.style.use("ggplot")
plt.figure()
# plt.plot(N, hist.history["loss"], label="train_loss")
# plt.plot(N, hist.history["classification_output_loss"], label="classification_output_loss")
# plt.plot(N, hist.history["segmentation_output_loss"], label="segmentation_output_loss")

# plt.plot(N, hist.history["classification_output_acc"], label="classification_output_acc")
# plt.plot(N, hist.history["segmentation_output_acc"], label="segmentation_output_acc")
# plt.plot(N, hist.history["segmentation_output_dice_coef"], label="segmentation_output_dice_coef")

plt.plot(N, hist.history["val_loss"], label="loss")
plt.plot(N, hist.history["val_classification_output_loss"], label="cla_loss")
plt.plot(N, hist.history["val_segmentation_output_loss"], label="seg_loss")

plt.plot(N, hist.history["val_classification_output_acc"], label="cla_acc")
plt.plot(N, hist.history["val_segmentation_output_acc"], label="seg_acc")
plt.plot(N, hist.history["val_segmentation_output_dice_coef"], label="Dice")

plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy/DSC")
plt.legend()
plt.savefig(os.path.join('output/', Name+'.png'))


NameError: name 'np' is not defined

In [17]:
print("------Start predicting------")
predictions_c, predictions_s = model.predict([test_x, test_x], batch_size=8)
print("------------------------------------------------ Classification testing ------------------------------------------------")

print(classification_report(test_c_y.argmax(axis=1), predictions_c.argmax(axis=1), target_names=lb.classes_, digits=4))

print("------------------------------------------------ Segmentation testing ------------------------------------------------")

# evaluate the model
# loss = model.evaluate(test_x, [test_c_y, test_s_y], verbose=0)
loss, cla_loss, seg_loss, cla_acc, seg_acc, seg_dice_coef = model.evaluate([test_x, test_x], [test_c_y, test_s_y], verbose=0)
print('Test total loss:', loss)
print('Test classification loss:', cla_loss)
print('Test segmentation loss:', seg_loss)

print('Test classification accuracy:', cla_acc)
print('Test segmentation accuracy:', seg_acc)
print('Test segmentation dice_coef:', seg_dice_coef)


preds_c, preds_s = model.predict([test_x, test_x], batch_size=8, verbose=1)
test_mask = test_s_y.flatten()
pred_mask = preds_s.flatten()
fpr, tpr, thresholds = roc_curve(test_mask, pred_mask, pos_label=1)
roc_auc = auc(fpr, tpr)
print("AUC:", roc_auc)

# Precision, Recall, accuracy, F1, IoU = p_r_f1_iou(test_s_y, preds_s)
# print('Precision:', Precision)
# print('Recall:', Recall)
# print('m_Iou:', IoU)
# print('F1_score:', F1)


------Start predicting------
------------------------------------------------ Classification testing ------------------------------------------------
                  precision    recall  f1-score   support

   bengin_images     0.8571    0.9767    0.9130        43
malignant_images     0.9333    0.6667    0.7778        21

        accuracy                         0.8750        64
       macro avg     0.8952    0.8217    0.8454        64
    weighted avg     0.8821    0.8750    0.8687        64

------------------------------------------------ Segmentation testing ------------------------------------------------
Test total loss: 0.6001038998365402
Test classification loss: 0.9678511321544647
Test segmentation loss: 0.2323567196726799
Test classification accuracy: 0.875
Test segmentation accuracy: 0.9539745151996613
Test segmentation dice_coef: 0.9520971179008484
64/64 [==============================] - 1s 11ms/step
AUC: 0.9878973282183112
Precision: 0.6899228282008861
Recall: 0.6181018